Task Events Inter-arrivals
==
(Run all the sections until the **Usage** section)
--

In [ ]:
import numpy
import math
from utils import math_utils as mu
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from utils import plot_utils as pu
from utils import output_utils as ou

output_notebook()

# Load and parse the job traces from the dataset
# Traces content
# 0: timestamp
# 1: missing info
# 2: job ID
# 3: task index
# 4: machine ID
# 5: event type
# 6: username
# 7: scheduling class
# 8: priority
# 9: resource req CPU cores
# 10: resource req RAM
# 11: resource req local disk space
# 12: different machine costraint

# Load all the traces inside the directory 
t_google_traces = sc.textFile("../task_events/*.gz")

t_google_traces_RDD = t_google_traces.map(lambda line: line.split(","))\
    .map(lambda tokens: (int(tokens[0]),int(tokens[2]),int(tokens[3]),tokens[4],tokens[5],tokens[6],tokens[7],tokens[8],tokens[9],tokens[10],tokens[11]))\
    .cache()
# Traces content
# 0: timestamp
# 1: job ID
# 2: task index
# 3: machine ID
# 4: event type
# 5: username
# 6: scheduling class
# 7: priority
# 8: resource req CPU cores
# 9: resource req RAM
# 10: resource req local disk space
# 11: different machine costraint

# The 2nd element of the trace was a "missing information", so for the sake of simplicity was cut out
print t_google_traces_RDD.take(1)

In [ ]:
# We remove from the traces those that occurred before the beginning of the trace window (timestamp = 0)
# and those that occured after the end of the trace window (timestamp = 2^63-1)
# and we sort them in ascending order wrt the timestamps
t_filtered_google_traces_RDD = t_google_traces_RDD.map(lambda elem: (elem[0],(elem[1],elem[2],elem[3],elem[4])))\
    .filter(lambda elem: elem[0] != 0 and elem[0] != (2^63 - 1))\
    .sortByKey(1,1)

# Traces content
# 0: timestamp
# 1.0: job ID
# 1.1: task index
# 1.2: machine ID
# 1.3: event type
print t_google_traces_RDD.count() - t_filtered_google_traces_RDD.count(), "Traces were removed"
print t_filtered_google_traces_RDD.take(1)

Main Functions
---

In [ ]:
# This function takes as input 
# event_type: int, corresponding to the event type found in the traces
# init_time: int (in seconds!), it is the time from which we want to evaluate the model
# finish_time: int (in seconds!), it is the time when we want to stop the evaluation
# granularity: int (in seconds!), define the level of granularity for plotting the results of the model
# E.G. over a window of 200 seconds we may have a granularity of 10 seconds
# which means that the derived traces will be clustered, based on the timestamps, in groups following
# that granularity
# For example assume to start from time = 0 till time = 200
# cluster 1: time interval 0-10
# cluster 2: time interval 10-20
# ...
# cluster 20: time interval 180-200

def t_eval_time_window(event_type, machine_ID, job_ID, init_time, finish_time, granularity):
    init_time, finish_time, granularity = mu.adjust_values(init_time, finish_time, granularity)
    
    # First of all we apply another filter to select only the traces that corresponde to the event_type in input  
    t_eval_traces_RDD = t_filtered_google_traces_RDD.filter(lambda elem: elem[0] >= init_time and elem[0] < finish_time)
    
    if(event_type != None):
        t_eval_traces_RDD = t_eval_traces_RDD.filter(lambda elem: elem[1][3] == unicode(event_type))
    if(machine_ID != None):
        t_eval_traces_RDD = t_eval_traces_RDD.filter(lambda elem: elem[1][2] == machine_ID)
    if(job_ID != None):
        t_eval_traces_RDD = t_eval_traces_RDD.filter(lambda elem: elem[1][0] == job_ID)
        
    t_eval_traces_RDD = t_eval_traces_RDD.map(lambda elem: elem[0])
    
    # Collect the RDD to get a python list
    t_eval_traces_list = t_eval_traces_RDD.collect()
    t_evaluated_means_list = []
    # This value will always contain the lowest bound for the clusterization based on the granularity
    # It's initial values is obviously the init_time
    t_lower_g = init_time
    # Define how many clusters we want to create depending on the input granularity
    t_n_cluster = (finish_time-init_time)/granularity
    
    t_interval_values_list = []
    # We iterate to creare each cluster
    for i in range(0,int(t_n_cluster+1)):
        # Define the cluster filtering the derived traces
        t_cluster_traces = [timestamp for timestamp in t_eval_traces_list if timestamp >= t_lower_g and timestamp < (t_lower_g+granularity)]
        # We then append to our list a tuple of this format
        # (cluster_lower_bound, cluster_upper_bound, mean_time_between_jobs)
        t_evaluated_means_list.append([t_lower_g, (t_lower_g+granularity), mu.mean_time_evaluation(sc, t_cluster_traces)])
        
        # interval_values_list contains all the inter-arrivals times between subsequent events among the intervals (clusters)
        t_interval_values_list.extend(mu.get_interval_values(sc, t_cluster_traces, t_lower_g, t_lower_g+granularity))
        # Increase the lower bound to reach the next cluster
        t_lower_g += granularity
    
    # Evaluate some metrics and plot them
    t_metrics_list = mu.evaluate_statistics(sc, t_interval_values_list)
    pu.plot_custom_metrics(sc, t_metrics_list, 0, "tasks", "time", "mean", "red")
    pu.plot_custom_metrics(sc, t_metrics_list, 1, "tasks", "time", "variance", "green")
    pu.plot_custom_metrics(sc, t_metrics_list, 2, "tasks", "time", "median", "blue")
    pu.plot_custom_metrics(sc, t_metrics_list, 3, "tasks", "time", "standard deviation", "orange")
    
    # Write on CSV
    #ou.write_csv(interval_values_list, event_type, init_time, finish_time, granularity)
    # Remove the NaNs and substitute them with 0s
    t_evaluated_means_list = mu.removeNans(t_evaluated_means_list)
    
    # Calculate the average time for the whole period
    t_mean_time_whole_period = mu.mean_time_evaluation(sc, t_eval_traces_RDD.collect())
    # Plot inter-irravals
    pu.plot_inter_arrivals(sc, t_evaluated_means_list, t_mean_time_whole_period)
    
# Evaluate a single day (24 h)
def t_eval_day(event_type, machine_ID, job_ID, day, granularity):
    init_time, finish_time = mu.get_day_function_parameters(day)
    # If it is None an error occurred
    if (init_time != None and finish_time != None):
        t_eval_time_window(event_type,machine_ID,job_ID,init_time,finish_time,granularity)

# Evaluate multiple days
def t_eval_days(event_type, machine_ID, job_ID, init_day, finish_day, granularity):
    init_time, finish_time = mu.get_days_function_parameters(init_day, finish_day)
    # If it is None an error occurred
    if (init_time != None and finish_time != None):
        t_eval_time_window(event_type,machine_ID,job_ID,init_time,finish_time,granularity)

Usage
---

In [ ]:
#t_eval_time_window(4, None, None, 600, 86400, 7200)
t_eval_day(1,None,None,2,7200)